## For Nile Data

In [1]:
lik_func= function(E,N){
    y=NULL
    for(i in 1:length(Nile)){
        y[i]=Nile[i]
    }
    a=NULL
    a[1]=0
    P=NULL
    P[1]=10^7
    v=NULL
    lik=0
    K=NULL
    Ft=NULL
    for(i in 1:length(Nile)){
        v[i]= y[i]- a[i]
        Ft[i]= P[i] + E
        K[i]= P[i]/Ft[i]
        a[i+1]= a[i]+ K[i]*v[i]
        P[i+1]= P[i]*(1-K[i])+ N
        lik= lik+log(Ft[i])+((v[i])^2)/Ft[i]
    }
    my_list= list(v=v,Ft=Ft,a=a,P=P,likelihood=(-(((length(Nile))/2)*log(2*pi))-(1/2)*lik))
    return(my_list)
}

## Kim and Nelson

In [51]:
library(zoo)
library(gdata)

data=read.xls('tvp.xls')
dint_lag=ts(data[,3],start= c(1959,03),frequency=4) 
inf_lag=ts(data[,4],start= c(1959,03),frequency=4)
surp_lag=ts(data[,5],start= c(1959,03),frequency=4)
m1gr_lag=ts(data[,6],start= c(1959,03),frequency=4)
m1gr=ts(data[,2],start= c(1959,03),frequency=4)

likelihood_function= function(E1,P1,P2,P3,P4,P5){
  Q=matrix(0,5,5)
  Q[1,1]=P1
  Q[2,2]=P2
  Q[3,3]=P3
  Q[4,4]=P4
  Q[5,5]=P5
  H=E1
  R=1
  Tt=1
  df=data.frame(year=as.Date(as.yearqtr(time(m1gr))),m1gr,dint_lag,inf_lag,surp_lag,m1gr_lag)
  
  y=c(0,length(m1gr))
  for (i in 1:length(m1gr)){
    y[i]=df[,2][i]
  }
  Z=array(1,c(row=1,col=5,length(m1gr)))
  for (i in 1:length(m1gr)){
    Z[1,2,i]=df[,3][i]
    Z[1,3,i]=df[,4][i]
    Z[1,4,i]=df[,5][i]
    Z[1,5,i]=df[,6][i]
  }
    
  a=array(0,c(5,1,length(m1gr)+1))
  P=array(0,c(5,5,length(m1gr)+1))
  P[1,1,1]=10^7
  P[2,2,1]=10^7
  P[3,3,1]=10^7
  P[4,4,1]=10^7
  P[5,5,1]=10^7  
  v=NULL
  Ft=NULL  
  K=array(0,c(5,1,length(m1gr)))
  lik=0
  for (i in 1:length(m1gr)){
      
    v[i]= y[i] - t(Z[,,i]) %*% a[,,i]
    Ft[i]= t(Z[,,i]) %*% P[,,i] %*% t(t(Z[,,i])) + H
    K[,,i]= Tt * P[,,i] %*% t(t(Z[,,i])) * (Ft[i])^(-1)
    a[,,i+1]= Tt * a[,,i] + K[,,i] * v[i]
    P[,,i+1]= Tt * P[,,i] %*% t(Tt - K[,,i] %*% t(Z[,,i])) + R * Q * (R) 
    lik= lik + log(abs(Ft[i]))+ v[i] * (Ft[i])^(-1) * v[i]
  }
    my_list=list(v=v,Ft=Ft,K=K,a=a,P=P,lik=lik)
    return(my_list)
    
    
  }

In [52]:
model1=likelihood_function(1,1,1,1,1,1)

In [53]:
xxx=NULL
for (i in 1:length(m1gr)){
    xxx=model1$P[,2,i]
}

In [54]:
model1$P

, , 1

      [,1]  [,2]  [,3]  [,4]  [,5]
[1,] 1e+07 0e+00 0e+00 0e+00 0e+00
[2,] 0e+00 1e+07 0e+00 0e+00 0e+00
[3,] 0e+00 0e+00 1e+07 0e+00 0e+00
[4,] 0e+00 0e+00 0e+00 1e+07 0e+00
[5,] 0e+00 0e+00 0e+00 0e+00 1e+07

, , 2

        [,1]    [,2]    [,3]    [,4]    [,5]
[1,] 4406847 8713575 8937301 9440685 5357682
[2,] 8713575 9704123 9755579 9871357 8932267
[3,] 8937301 9755579 9798088 9893730 9117960
[4,] 9440685 9871357 9893730 9944069 9535768
[5,] 5357682 8932267 9117960 9535768 6146877

, , 3

         [,1]     [,2]     [,3]     [,4]     [,5]
[1,] 31561413 29220216 29098595 28824949 31044524
[2,] 39341028 35964605 35789205 35394559 38595583
[3,] 39745163 36314962 36136770 35735837 38987846
[4,] 40654469 37103267 36918789 36503715 39870438
[5,] 33278989 30709237 30575743 30275383 32711642

, , 4

          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 124823708 118992259 118689326 118007729 123536245
[2,] 154327541 147131823 146758019 145916961 152738877
[3,] 155860207 148593618 148216134 147366791 154255897
[4,] 159308708 151882659 151496890 150628911 157669192
[5,] 131337541 125204891 124886312 124169509 129983582

, , 5

          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 474971187 444463469 442878651 439312813 468235716
[2,] 587287423 549569180 547609789 543201162 578960019
[3,] 593122032 555029216 553050368 548597958 584711930
[4,] 606249907 567314302 565291672 560740756 597653735
[5,] 499768285 467668633 466001117 462249209 492681350

, , 6

           [,1]       [,2]       [,3]       [,4]       [,5]
[1,] 1765274447 1645955632 1639757250 1625810892 1738931325
[2,] 2182721369 2035187465 2027523362 2010279135 2150148940
[3,] 2204406918 2055407293 2047667052 2030251506 2171510889
[4,] 2253199428 2100901935 2092990375 2075189366 2219575298
[5,] 1857438082 1731889961 1725367979 1710693519 1829719660

, , 7

           [,1]       [,2]       [,3]       [,4]       [,5]
[1,] 6543019940 6100167280 6077161952 6025399939 6445247242
[2,] 8090298006 7542720901 7514275342 7450272806 7969404320
[3,] 8170676064 7617658728 7588930562 7524292154 8048581287
[4,] 8351526795 7786268937 7756904898 7690835781 8226729565
[5,] 6884626897 6418653249 6394446831 6339982364 6781749566

, , 8

            [,1]        [,2]        [,3]        [,4]        [,5]
[1,] 25448558689 24090600321 24020057050 23861334601 25148749600
[2,] 31466575342 29787490124 29700264942 29504008176 31095868094
[3,] 31779199491 30083432362 29995340590 29797133989 31404809223
[4,] 32482604222 30749302777 30659261169 30456667441 32099927155
[5,] 26777212154 25348355627 25274129335 25107120086 26461750221

, , 9

             [,1]         [,2]         [,3]         [,4]         [,5]
[1,]  98643385185  92985439788  92691520633  92030202164  97394227997
[2,] 121970346204 114974422888 114610998406 113793292865 120425791196
[3,] 123182136027 116116707384 115749672240 114923842696 121622235683
[4,] 125908664667 118686848952 118311689805 117467581251 124314237296
[5,] 103793495159  97840151942  97530887451  96835041954 102479120250

, , 10

             [,1]         [,2]         [,3]         [,4]         [,5]
[1,] 375649796144 352452520811 351247467904 348536097336 370528318083
[2,] 464482596603 435799682915 434309661865 430957112616 458150003182
[3,] 469097286168 440129404337 438624579754 435238722530 462701777681
[4,] 479480343540 449871283028 448333150471 444872350267 472943276078
[5,] 395262239073 370853848379 369585880408 366732950863 389873371820

, , 11

             [,1]         [,2]         [,3]         [,4]         [,5]
[1,] 1.450902e+12 1.368723e+12 1.364454e+12 1.354849e+12 1.432759e+12
[2,] 1.794008e+12 1.692396e+12 1.687117e+12 1.675240e+12 1.771574e+12
[3,] 1.811832e+12 1.709210e+12 1.703879e+12 1.691884e+12 1.789175e+12
[4,] 1.851935e+12 1.747042e+12 1.741593e+12 1.729332e+12 1.828777e+12
[5,] 1.526653e+12 1.440183e+12 1.435691e+12 1.425585e+12 1.507562e+12

, , 12

             [,1]         [,2]         [,3]         [,4]         [,5]
[1,] 5.598950e+12 5.27

In [55]:
faa=function(xx){
    likelihood_function(xx[1],xx[2],xx[3],xx[4],xx[5],xx[6])
}

In [56]:
optim(c(0.01,0.01,0.01,0.01,0.01,0.01),faa,control=list(fnscale= -1))

ERROR: Error in optim(c(0.01, 0.01, 0.01, 0.01, 0.01, 0.01), faa, control = list(fnscale = -1)): (list) object cannot be coerced to type 'double'


In [57]:
optim(c(1,0.01,1,0.01,1,0.01),faa,control=list(fnscale= -1))

ERROR: Error in optim(c(1, 0.01, 1, 0.01, 1, 0.01), faa, control = list(fnscale = -1)): (list) object cannot be coerced to type 'double'


In [31]:
optim(c(0.121,0.012,0.0008,0.07,0.0006,0.001),faa,control=list(fnscale= -1))

ERROR: Error in optim(c(0.121, 0.012, 8e-04, 0.07, 6e-04, 0.001), faa, control = list(fnscale = -1)): (list) object cannot be coerced to type 'double'


In [46]:
optim(c(10,0,0,0,0,0),faa,control=list(fnscale= -1))

$par
[1] 10.005211857 -0.032971508  0.208965233  0.008962984  0.665312333
[6]  0.040623263

$value
[1] 15023.61

$counts
function gradient 
     501       NA 

$convergence
[1] 1

$message
NULL


In [50]:
likelihood_function(0.126284,.012133,.000896,.074544,.000683,.001184)

$v
  [1]   0.75000000  -0.87000000  -0.07038455  -0.07038494   1.23311756
  [6]  -0.23290201   0.56303146   0.53529007   0.37822434  -1.94574688
 [11]   0.20322520  -0.65967569  -0.75584038   0.65434096   0.45743009
 [16]  -0.28172918   0.35963554  -1.27575164   0.12697887  -0.20150693
 [21]   1.28677989  -0.64220803  -1.73141820   0.06748889   0.15038894
 [26]   1.19715234  -1.20225257  -1.66086116  -1.29685477   0.54519511
 [31]   0.75171366   0.91025833   0.19373806  -2.25859854  -0.13160508
 [36]   0.20811833   0.37512371  -0.34792083  -0.26385472  -0.96589936
 [41]  -0.54679226   0.35392770   0.29008920  -0.43208989   0.71564932
 [46]   0.66455442  -0.44203992   1.01211663  -0.66362434  -0.88213184
 [51]   1.31797519  -0.94124998   0.88395169  -0.40658339  -0.75236780
 [56]  -1.69920309  -0.34791669   0.05868684   0.18895131  -1.11230312
 [61]   0.10377007   0.21928356  -0.56131043   1.06839932   0.87949558
 [66]  -2.09990320   0.58608962   0.78237460  -0.19156332   0.10216069
 [71]   0.49035150  -1.19857882   0.03695618   0.74603284  -0.27434153
 [76]  -0.01121544  -0.89081035  -0.23818181  -0.58774874   0.54377280
 [81]   1.04321566  -2.24338999  -0.02984936  -2.81969412   6.03946883
 [86]  -0.20494107  -3.53955995   1.24155787   0.35966835  -0.92826384
 [91]   1.09045293  -0.68824694   1.50777963   1.34553462   1.49687716
 [96]   1.37910241 -10.37147115  -0.79467995   0.02681931  -0.45274522
[101]  -0.17796712   0.09517054   1.69859761  -0.19585550   0.15201754
[106]   0.60687779

$Ft
  [1]  1.262840e-01  1.579814e-01  2.452543e-01  2.093528e-01  1.024284e+00
  [6]  3.117997e+00  2.214816e+01  1.117596e+02  1.548088e+02  3.657099e+02
 [11]  2.389754e+03  1.326510e+04  4.896735e+04  8.804045e+04  6.500863e+05
 [16]  4.631330e+06  1.225092e+07  1.020610e+08  2.277762e+08  7.712905e+08
 [21]  1.449279e+09  1.308913e+10  7.252118e+10  1.680413e+11  8.277150e+11
 [26]  2.498282e+12  2.412493e+13  1.543986e+14  4.188352e+14  9.687014e+14
 [31]  5.568992e+15  6.985815e+15  6.760334e+16  1.740645e+18  5.136373e+18
 [36]  2.276703e+19  1.079719e+20  5.446789e+20  3.387519e+21  1.428379e+22
 [41]  4.828776e+22  2.599289e+23  1.083430e+24  5.400550e+24  1.360704e+25
 [46]  7.217357e+25  3.724959e+26  6.489636e+26  9.410784e+27  4.115315e+28
 [51]  3.415348e+28  5.062868e+29  1.828889e+30  1.353747e+31  6.678495e+31
 [56]  4.004210e+32  2.055525e+33  1.249819e+34  3.269504e+34  3.539414e+35
 [61]  1.185958e+36  5.913350e+36  2.542391e+37  3.226363e+37  1.435424e+38
 [66]  2.474044e+39  3.293196e+39  1.364530e+40  7.458844e+40  4.027411e+41
 [71]  2.316980e+42  1.887987e+43  7.059826e+43  2.555152e+44  1.326631e+45
 [76]  5.978944e+45  4.248904e+46  2.225111e+47  1.049595e+48  4.215050e+48
 [81]  2.765310e+49  1.731493e+50  6.932883e+50  3.665667e+51 -6.072007e+51
 [86]  5.065277e+52  4.708693e+53  6.055399e+53  3.299972e+54  1.793547e+55
 [91] -5.087238e+54  1.548540e+56  2.148149e+56  1.072388e+57  4.620692e+57
 [96]  1.178691e+58  7.781654e+58  5.306903e+59  7.782525e+59  8.222001e+60
[101]  2.821614e+61  7.669064e+61 -7.445667e+61  2.396970e+63  9.267849e+63
[106]  4.354260e+64

$K
, , 1

     [,1]
[1,]    0
[2,]    0
[3,]    0
[4,]    0
[5,]    0

, , 2

             [,1]
[1,] 0.0768001966
[2,] 0.0030626397
[3,] 0.2359265580
[4,] 0.0001729318
[5,] 0.0056209161

, , 3

            [,1]
[1,] 0.112400878
[2,] 0.007534334
[3,] 0.605163785
[4,] 0.004292262
[5,] 0.002204004

, , 4

           [,1]
[1,] 0.25849635
[2,] 0.01385783
[3,] 1.32786898
[4,] 0.01301457
[5,] 0.02022085

, , 5

           [,1]
[1,] 0.18019016
[2,] 0.01330684
[3,] 1.18332193
[4,] 0.01083270
[5,] 0.01865653

, , 6

           [,1]
[1,] 0.73151566
[2,] 0.05622676
[3,] 4.68034969
[4,] 0.04444632
[5,] 0.07679936

, , 7

           [,1]
[1,] 0.19322973
[2,] 0.01521430
[3,] 1.25258488
[4,] 0.01188510
[5,] 0.02027256

, , 8

           [,1]
[1,] 0.40329968
[2,] 0.03139760
[3,] 2.59739211
[4,] 0.02467306
[5,] 0.04238553

, , 9

           [,1]
[1,] 1.22306786
[2,] 0.09

In [2]:
Z

ERROR: Error in eval(expr, envir, enclos): object 'Z' not found


In [47]:
aa=array(1,c(5,5))

In [48]:
aa

1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
